In [1]:
import sys

sys.path.append('../../..')

In [2]:
from batchflow.research import Research

def power(a=2, b=3):
    return a ** b

research = Research().add_callable(power, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  5.34it/s]


,id,iteration,power
0,b8c9f03196043317,0,8


In [3]:
research = Research().add_callable(power, a=3, b=2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  4.41it/s]


,id,iteration,power
0,b8c9f03198368093,0,9


In [4]:
research = Research().add_callable(power, a=3, b=2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  3.34it/s]


,id,iteration,power
0,b8c9f03152565960,0,9


In [5]:
research = Research().power(3, 2, save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


,id,iteration,power
0,b8c9f03159121612,0,9


In [6]:
from batchflow.research import Domain, EC

domain = Domain({'a': [2, 3], 'b': [2, 3, 4]})

research = Research(domain=domain).power(a=EC('a'), b=EC('b'), save_to='power')
research.run(dump_results=False)

research.results.df

0it [00:01, ?it/s]
0it [00:01, ?it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
0it [00:02, ?it/s]/6 [00:00<00:00,  5.90it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 6/6 [00:00<00:00,  6.20it/s]



,id,a,b,iteration,power
0,bef0db4916169678,2,2,0,4
1,984d62c908336512,2,3,0,8
2,270aa80393525847,2,4,0,16
3,5420406379778025,3,2,0,9
4,72b9965c29496210,3,3,0,27
5,4b4a714538847906,3,4,0,81


In [7]:
from batchflow.research import Domain, EC

domain = Domain({'a': [2, 3]}) * Domain({'b': [2, 3, 4]})

research = Research(domain=domain).power(a=EC('a'), b=EC('b'), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 6/6 [00:00<00:00, 11.62it/s]


,id,a,b,iteration,power
0,bef0db4974587248,2,2,0,4
1,984d62c970751655,2,3,0,8
2,270aa80343544640,2,4,0,16
3,5420406376862851,3,2,0,9
4,72b9965c44087826,3,3,0,27
5,4b4a714519541959,3,4,0,81


In [8]:
domain = Domain({'a': [2, 3]}) + Domain({'b': [2, 3, 4]})

research = Research(domain=domain).power(kwargs=EC(), save_to='power')
research.run(dump_results=False)

research.results.df

100%|██████████| 5/5 [00:00<00:00,  8.89it/s]


,id,a,b,iteration,power
0,71d4879f28582754,2.0,NaN,0,8
1,e32d7dce20288324,3.0,NaN,0,27
2,ff973b7658739419,NaN,2.0,0,4
3,6bffa3ee91968203,NaN,3.0,0,8
4,05064ba246167129,NaN,4.0,0,16


In [9]:
from batchflow.research import O

domain = Domain({'a': [2, 3], 'b': [3]})

def inc(x):
    return (x + i for i in range(2))

research = (Research(domain=domain)
            .add_callable(power, a=EC('a'), b=EC('b'))
            .add_generator(inc, save_to='inc', x=O('power'))
           )
research.run(dump_results=False)

research.results.df

100%|██████████| 2/2 [00:00<00:00,  4.58it/s]


,id,a,b,iteration,inc
0,984d62c974571040,2,3,0,8
1,984d62c974571040,2,3,1,9
2,984d62c974571040,2,3,2,9
3,72b9965c75179628,3,3,0,27
4,72b9965c75179628,3,3,1,28
5,72b9965c75179628,3,3,2,28


In [10]:
research = (Research(domain=domain)
            .add_callable(power, a=EC('a'), b=EC('b'))
            .add_generator(inc, save_to='inc', x=O('power'))
            .add_callable('root', power, a=O('inc'), b=0.5, save_to='root',
                          when='last')
           )
research.run(dump_results=False, n_iters=2)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  7.17it/s]


,id,a,b,iteration,inc,root
0,984d62c943416619,2,3,0,8,NaN
1,984d62c943416619,2,3,1,9,3.000000
2,72b9965c39028980,3,3,0,27,NaN
3,72b9965c39028980,3,3,1,28,5.291503


In [11]:
research.results.to_df(pivot=False)

,id,a,b,iteration,name,value
0,984d62c943416619,2,3,0,inc,8.000000
1,984d62c943416619,2,3,1,inc,9.000000
2,984d62c943416619,2,3,1,root,3.000000
3,72b9965c39028980,3,3,0,inc,27.000000
4,72b9965c39028980,3,3,1,inc,28.000000
5,72b9965c39028980,3,3,1,root,5.291503


In [12]:
class MyCalc:
    def __init__(self, b):
        self.b = b
    
    def power(self, a):
        return a ** self.b

research = (Research(domain=domain)
            .add_instance('calc', MyCalc, b=EC('b'))
            .add_callable('calc.power', a=EC('a'))
            .add_generator(inc, x=O('calc.power'), save_to='inc')
           )

research.run(n_iters=2, dump_results=False)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  6.73it/s]


,id,a,b,iteration,inc
0,984d62c998268508,2,3,0,8
1,984d62c998268508,2,3,1,9
2,72b9965c72948110,3,3,0,27
3,72b9965c72948110,3,3,1,28


In [13]:
def heavy_callable():
    i = int(2 * 10e1)
    for i in range(i):
        pass

In [14]:
%%timeit

research = Research(n_reps=2).add_callable(heavy_callable)
research.run(dump_results=False)

100%|██████████| 2/2 [00:00<00:00,  3.09it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.76it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.61it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.81it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  2.99it/s]


848 ms ± 44.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit

research = Research(n_reps=2).add_callable(heavy_callable)
research.run(dump_results=False, workers=2)

100%|██████████| 2/2 [00:00<00:00,  3.35it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  3.47it/s]
0it [00:15, ?it/s]
0it [00:15, ?it/s]
0it [00:14, ?it/s]
0it [00:13, ?it/s]
0it [00:02, ?it/s]
0it [00:01, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  5.63it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  5.62it/s]
0it [00:01, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  6.00it/s]


511 ms ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
import numpy as np
from batchflow.research import E

def load_data(random):
    return random.normal(size=10)

def mean(array, use_numpy=True):
    if use_numpy:
        return f"{np.mean(array):.02f} (with numpy)"
    return f"{sum(array) / len(array):.02f} (without numpy)"

domain = Domain({'use_numpy': [False, True]})

research = (Research(domain=domain)
            .add_callable(load_data, random=E().random)
            .add_callable(mean, array=O('load_data'), use_numpy=EC('use_numpy'),
                          save_to='stat')
           )

research.remove('research', ask=False)
research.run(n_iters=2)

research.results.df

0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  9.79it/s]


,id,use_numpy,iteration,stat
0,7635c7e495587543,False,0,-0.23 (without numpy)
1,7635c7e495587543,False,1,-0.19 (without numpy)
2,8e95395710053729,True,0,-0.31 (with numpy)
3,8e95395710053729,True,1,-0.19 (with numpy)


In [17]:
research = (Research(domain=domain)
            .add_callable(load_data, random=E().random, root=True)
            .add_callable(mean, array=O('load_data'), use_numpy=EC('use_numpy'),
                          save_to='stat')
           )

research.remove('research', ask=False)
research.run(n_iters=2, branches=2)

research.results.df

100%|██████████| 4/4 [00:00<00:00,  9.15it/s]


,id,use_numpy,iteration,stat
0,7635c7e441130185,False,0,0.06 (without numpy)
1,7635c7e441130185,False,1,-0.10 (without numpy)
2,8e95395769090980,True,0,0.06 (with numpy)
3,8e95395769090980,True,1,-0.10 (with numpy)
